In [21]:
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.models import Sequential
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import time

%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set()

import warnings
warnings.filterwarnings('ignore')


In [41]:
df = pd.read_csv('../FeatureEngineering/MetaData/data6_&_odds.csv')
df.dropna(inplace=True)

In [42]:
train_data = df.loc[(df.season <= 2013) & (df.season >= 2007)]
valid_data = df.loc[(df.season > 2013) & (df.season < 2016)]
test_data = df.loc[df.season >= 2016]
full_train_data = pd.concat([train_data, valid_data], axis=0)

X, y = train_data.drop(columns=['home_team_wins']), train_data.home_team_wins
valid_X, valid_y = valid_data.drop(columns=['home_team_wins']), valid_data.home_team_wins
test_X, test_y = test_data.drop(columns=['home_team_wins']), test_data.home_team_wins

# Split our data
X_train, y_train = train_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), train_data.home_team_wins
X_val, y_val = valid_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), valid_data.home_team_wins
X_test, y_test = test_data.drop(columns=["game_date_est","season","game_id","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"]), test_data.home_team_wins


In [58]:
arr_X_train = X_train.to_numpy()
arr_X_train = arr_X_train.reshape(arr_X_train.shape[0], arr_X_train.shape[1], 1)

arr_X_val = X_val.to_numpy()
arr_X_val = arr_X_val.reshape(arr_X_val.shape[0], arr_X_val.shape[1], 1)

### Making a LSTM model

In [60]:
model = Sequential()
model.add(LSTM(90, activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(90, activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])




In [48]:
# model = get_model(input_shape=(100,1))

In [61]:
model.fit(arr_X_train, y_train, epochs=100, batch_size=32)
# evaluate the keras model
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
216/216 [==============================] - 38s 162ms/step - loss: 0.6815 - accuracy: 0.5846
Epoch 2/100
216/216 [==============================] - 38s 175ms/step - loss: 0.6771 - accuracy: 0.5935
Epoch 3/100
216/216 [==============================] - 34s 159ms/step - loss: 0.6750 - accuracy: 0.5967
Epoch 4/100
216/216 [==============================] - 37s 171ms/step - loss: 0.6695 - accuracy: 0.6004
Epoch 5/100
216/216 [==============================] - 33s 154ms/step - loss: 0.6652 - accuracy: 0.6076
Epoch 6/100
216/216 [==============================] - 32s 147ms/step - loss: 0.6630 - accuracy: 0.6140
Epoch 7/100
216/216 [==============================] - 32s 150ms/step - loss: 0.6610 - accuracy: 0.6163
Epoch 8/100
216/216 [==============================] - 33s 154ms/step - loss: 0.6599 - accuracy: 0.6183
Epoch 9/100
216/216 [==============================] - 37s 171ms/step - loss: 0.6590 - accuracy: 0.6204
Epoch 10/100
216/216 [==============================] - 40s 184m

In [76]:
start_time = time.time()
# val_score = model.score(arr_X_val, y_val)

preds = model.predict(arr_X_val)
preds = (preds > 0.7)

# test_score = model.score(X_val, y_val)

76/76 [==============================] - 6s 74ms/step
